In [1]:
import setup
from transformers import BertTokenizer, BertModel
import torch

setup.init_django()
from decouple import config

/home/nazk33r/Documents/LLM/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)


In [3]:
from blog.models import BlogPost, EMBEDDING_MODEL

In [4]:
RECREATE_DATE = True

In [5]:
docs = [
    "The brown dog barked loudly at the passing car.",
    "A small cat climbed up the tall green tree.",
    "The man walked quickly to catch the morning train.",
    "A woman in red dress danced gracefully under the moonlight."
]


In [6]:
new_data = []
for i, x in enumerate(docs):
    new_data.append(BlogPost(title=f"Blog Post {i+1}", content=x, can_delete=True))

if RECREATE_DATE:
    qs = BlogPost.objects.filter(can_delete=True)
    qs.delete()
    BlogPost.objects.bulk_create(new_data)

In [7]:
BlogPost.objects.all().count()

4

In [8]:
def get_embedding(text):
    text = text.replace("\n", " ").strip()
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)
    # Get the mean of the embeddings for the tokens in the sentence
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    return embedding


In [9]:
for obj in qs:
    if obj.embedding is None:
        obj.embedding = get_embedding(obj.get_embedding_text_raw())
        obj.save()


In [10]:
# query = "The brown dog barked loudly at the passing car."
query = "A woman in red dress danced gracefully under the moonlight."
query_embedding = get_embedding(query)


In [11]:
from pgvector.django import CosineDistance
from django.db.models import F, FloatField
qs = BlogPost.objects.annotate(
    distance=CosineDistance('embedding',query_embedding),
    similarity=1 - F("distance")
    ).order_by('distance')[:2]
for obj in qs:
    print(obj.title, obj.distance,obj.similarity * 100)

Blog Post 4 0.0 100.0
Blog Post 2 0.27707501523614475 72.29249847638553


In [16]:
from products.models import Embedding

In [19]:
from pgvector.django import CosineDistance
from django.db.models import F, FloatField
qs = Embedding.objects.annotate(
    distance=CosineDistance('embedding',query_embedding),
    similarity=1 - F("distance")
    ).order_by('distance')[:2]
for obj in qs:
    print(obj.object_id, obj.content_object, obj.content_object.title, obj.distance, obj.similarity * 100)

AttributeError: 'NoneType' object has no attribute 'title'